In [1]:
import ast


In [2]:
area_info = {}
net_features = []

with open('/content/drive/MyDrive/nets.txt', 'r') as f:
    lines = f.readlines()

# First, parse the area information
area_lines = []
i = 0
for i, line in enumerate(lines):
    line = line.strip()
    if line.startswith("### Design Area Information ###"):
        continue
    elif line.startswith("die_width") or "=" in line:
        key, val = line.split("=")
        area_info[key.strip()] = int(val.strip())
    elif line.startswith("{"):  # Stop parsing area info when the first net line appears
        break

# Then, parse the net feature lines starting from i (where first dict line appears)
for line in lines[i:]:
    line = line.strip()
    if line:
        try:
            net = ast.literal_eval(line)
            net_features.append(net)
        except Exception as e:
            print(f"Error parsing line: {line}\n{e}")

In [3]:
node_features = {}
for net in net_features:
    driver = net['driver']
    node_features[driver['id']] = [
        driver['x'], driver['y'], int(driver['is_fixed']), driver['area'], driver['cell_type_onehot']
    ]
    for sink in net['sinks']:
        node_features[sink['id']] = [
            sink['x'], sink['y'], int(sink['is_fixed']), sink['area'], sink['cell_type_onehot']
        ]


In [4]:
node_features[56]

[36480,
 36400,
 0,
 3192000,
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [5]:
sorted_node_ids = sorted(node_features.keys())
node_id_to_index = {nid: idx for idx, nid in enumerate(sorted_node_ids)}

# Node feature matrix
import torch
X = torch.tensor([node_features[nid][:4] + node_features[nid][4] for nid in sorted_node_ids], dtype=torch.float)


# Create hyperedges
hyperedges = []
for net in net_features:
    node_ids = [net['driver']['id']] + [sink['id'] for sink in net['sinks']]
    edge = [node_id_to_index[nid] for nid in node_ids]
    hyperedges.append(edge)

In [6]:
X[0]

tensor([6.6270e+04, 1.4000e+02, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00])

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HyperGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(HyperGNN, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)  # Output: (x, y) positions

    def forward(self, x):
        x = F.relu(self.fc1(x))
        pos = self.fc2(x)
        return pos  # Shape: [num_nodes, 2]

In [8]:
def soft_density_loss(x, y, area_info, cell_area=100000, bin_size=50, density_threshold=0.4, sigma=1000):
    """
    Soft differentiable binwise density loss using Gaussian smearing.
    """
    lx, ux = area_info['lx'], area_info['ux']
    ly, uy = area_info['ly'], area_info['uy']

    device = x.device

    x = torch.clamp(x, lx, ux)
    y = torch.clamp(y, ly, uy)

    # Create bin center coordinates
    x_bins = torch.arange(lx + bin_size/2, ux, bin_size, device=device)
    y_bins = torch.arange(ly + bin_size/2, uy, bin_size, device=device)
    x_centers, y_centers = torch.meshgrid(x_bins, y_bins, indexing='ij')  # (Bx, By)

    #print(x_centers, y_centers)

    Bx, By = x_centers.shape
    num_cells = x.shape[0]

    # Flatten bin centers for easier computation
    x_centers_flat = x_centers.flatten().unsqueeze(0)  # (1, B)
    y_centers_flat = y_centers.flatten().unsqueeze(0)  # (1, B)

    # Expand cell coords
    x_expand = x.unsqueeze(1)  # (N, 1)
    y_expand = y.unsqueeze(1)  # (N, 1)

    # Gaussian contribution from each cell to each bin
    dx2 = (x_expand - x_centers_flat) ** 2
    dy2 = (y_expand - y_centers_flat) ** 2
    gauss = torch.exp(-(dx2 + dy2) / (2 * sigma**2))  # (N, B)
    #print(gauss)
    #print("gauss max:", gauss.max().item(), "min:", gauss.min().item())
    #print("x range:", x.min().item(), x.max().item())
    #print("bin range:", x_centers.min().item(), x_centers.max().item())


    # Sum over cells, scale by cell area
    density_per_bin = torch.sum(gauss, dim=0) * cell_area  # (B,)

    #print(density_per_bin)

    bin_area = bin_size * bin_size
    density_norm = density_per_bin / bin_area  # (B,)

    # Compute penalty for bins over threshold
    penalty = torch.clamp(density_norm - density_threshold, min=0.0)
    loss = penalty.sum()
    #print (loss)

    return loss

In [9]:
def smooth_hpwl(positions, hyperedges, alpha=10.0):
    """
    Compute smooth half-perimeter wirelength (HPWL) over all nets using log-sum-exp.

    Args:
        positions: Tensor of shape (num_nodes, 2) with (x, y) positions
        hyperedges: List of lists, where each sublist contains indices of nodes in a net
        alpha: Smoothing factor for log-sum-exp

    Returns:
        Scalar tensor representing total smooth HPWL
    """
    hpwl_list = []

    for net in hyperedges:
        pins = positions[net]  # Shape: (num_pins_in_net, 2)
        x = pins[:, 0]
        y = pins[:, 1]

        max_x = (1.0 / alpha) * torch.logsumexp(alpha * x, dim=0)
        min_x = -(1.0 / alpha) * torch.logsumexp(-alpha * x, dim=0)
        max_y = (1.0 / alpha) * torch.logsumexp(alpha * y, dim=0)
        min_y = -(1.0 / alpha) * torch.logsumexp(-alpha * y, dim=0)

        hpwl = (max_x - min_x) + (max_y - min_y)
        hpwl_list.append(hpwl)

    total_hpwl = torch.stack(hpwl_list).sum()
    return total_hpwl

In [10]:

import torch

def validity_penalty(x_pred, y_pred, area_info):
    penalty = 0

    # Check validity constraints for x_pred (within lx and ux bounds)
    penalty += torch.sum(torch.maximum(torch.tensor(0.0), x_pred - area_info['ux'])) + \
               torch.sum(torch.maximum(torch.tensor(0.0), area_info['lx'] - x_pred))

    # Check validity constraints for y_pred (within ly and uy bounds)
    penalty += torch.sum(torch.maximum(torch.tensor(0.0), y_pred - area_info['uy'])) + \
               torch.sum(torch.maximum(torch.tensor(0.0), area_info['ly'] - y_pred))

    return penalty

In [11]:
import torch

def overlap_penalty(x_pred, y_pred, area_info, overlap_threshold=0.5):
    """
    Calculate the overlap penalty for cell placements. If any cells overlap based on the threshold, apply a penalty.

    Args:
        x_pred: Tensor of predicted x positions of cells.
        y_pred: Tensor of predicted y positions of cells.
        area_info: Dictionary containing the area of each cell.
        overlap_threshold: Threshold for determining if two cells are overlapping.

    Returns:
        overlap penalty: The penalty for overlapping cells.
    """
    penalty = 0
    num_cells = x_pred.shape[0]  # Number of cells

    for i in range(num_cells):
        for j in range(i + 1, num_cells):
            # Calculate distance between cells (i, j)
            dist_x = x_pred[i] - x_pred[j]
            dist_y = y_pred[i] - y_pred[j]

            # Calculate Euclidean distance
            distance = torch.sqrt(dist_x ** 2 + dist_y ** 2)

            # Calculate the sum of the areas of the two cells
            area_sum = 20000  # Assuming area_info is a tensor or list with cell areas

            # If the distance between the cells is less than the threshold, apply a penalty
            if distance < overlap_threshold * area_sum:
                penalty += (overlap_threshold * area_sum - distance) ** 2  # Square the penalty for sharper gradient

    return penalty

In [12]:
from sklearn.cluster import KMeans
import torch

def clustering(x_pred, y_pred, num_clusters=10):
    """
    Apply K-Means clustering to group cells into clusters.
    Args:
        x_pred: Tensor of predicted x positions
        y_pred: Tensor of predicted y positions
        num_clusters: Number of clusters to use
    Returns:
        cluster_labels: Cluster labels for each cell
    """
    positions = torch.stack([x_pred, y_pred], dim=1) # Convert tensor to numpy array for KMeans
    positions_np = positions.detach().numpy()

    kmeans = KMeans(n_clusters=num_clusters)
    cluster_labels = kmeans.fit_predict(positions_np)  # Cluster labels for each cell
    return torch.tensor(cluster_labels, dtype=torch.long)

def clustering_penalty(cluster_labels, num_clusters):
    """
    Compute penalty for clustering, based on how evenly distributed the clusters are.
    Args:
        cluster_labels: Tensor of cluster labels for each cell
        num_clusters: Number of clusters
    Returns:
        penalty: The penalty term based on clustering
    """
    # Count the number of cells in each cluster
    cluster_counts = torch.bincount(cluster_labels, minlength=num_clusters).float()

    # Calculate the ideal number of cells per cluster
    ideal_count = len(cluster_labels) / num_clusters

    # Compute the clustering penalty: sum of squared differences from ideal count
    penalty = torch.sum((cluster_counts - ideal_count) ** 2)
    return penalty


In [13]:
def total_loss(x_pred, y_pred, hyperedges, area_info, cell_area,
               lambda_validity=1.0, overlap_threshold=0.0, lambda_hpwl=1.0, lambda_density=1.0,
               alpha=10.0, bin_size=200, density_threshold=0.4):
    positions = torch.stack([x_pred, y_pred], dim=1)

    hpwl = smooth_hpwl(positions, hyperedges, alpha)
    density = soft_density_loss(x_pred, y_pred, area_info, cell_area, bin_size, density_threshold)
    validity = validity_penalty(x_pred, y_pred, area_info)

    print("x range:", x_pred.min().item(), x_pred.max().item())
    print("y range:", y_pred.min().item(), y_pred.max().item())

    # Loss shaping to reward good performance
    hpwl_term = torch.log1p(hpwl)
    density_term = torch.clamp(torch.log1p(1 + density), max=10.0)
    validity_term = torch.clamp(torch.log1p(1 + validity), max=20.0)

    # You can subtract a small reward if validity is near zero (clipped)
    validity_reward = 1.0 / (1.0 + validity_term)  # Range ~ (0, 1]

    lambda_validity = 1000.0
    lambda_hpwl = 1.0
    lambda_density = 10.0

    # Weighted loss
    total_loss_value = (
        lambda_hpwl * hpwl_term +
        lambda_density * density_term +
        lambda_validity * validity_term
    )

    return total_loss_value

In [14]:
def old_loss(
    x_pred,
    y_pred,
    hyperedges,
    area_info,
    cell_area,
    overlap_threshold=0.2,
    bin_size=200,
    density_threshold=0.8,
    alpha=10.0,
    #lambda_hpwl=1.0,
    #lambda_density=8000.0,
    #lambda_validity_max=1e6,
    #warmup_epochs=20,
):
    # Combine positions
    positions = torch.stack([x_pred, y_pred], dim=1)
    print("x range:", x_pred.min().item(), x_pred.max().item())
    print("y range:", y_pred.min().item(), y_pred.max().item())
    # Compute raw HPWL and log-scaled version
    raw_hpwl = smooth_hpwl(positions, hyperedges, alpha)
    #log_hpwl = torch.log(raw_hpwl + 1)

    # Compute density loss
    density_loss_term = soft_density_loss(x_pred, y_pred, area_info, cell_area, bin_size, density_threshold)

    # Compute validity penalty (soft, not clamped)
    validity_penalty_term = validity_penalty(x_pred, y_pred, area_info)
    #log_validity = torch.log(validity_penalty_term + 1)

    # Linear warm-up schedule
    #hpwl_weight = lambda_hpwl * min(1.0, epoch / warmup_epochs)
    #validity_weight = lambda_validity_max * min(1.0, epoch / warmup_epochs)

    # Base loss (gradually increase hpwl contribution)
    base_loss = 1 * raw_hpwl + 0.5 * density_loss_term

    # Total loss
    total_loss_value = base_loss + 10000*validity_penalty_term

    return total_loss_value

In [15]:
def train(model, X, hyperedges, area_info, cell_area, grid_size,
          lambda_validity=1.0, lambda_overlap=0.1, lambda_hpwl=1.0, lambda_density=1.0,
          overlap_threshold=0.5, bin_size=500, density_threshold=0.8,
          num_epochs=1, lr=0.03):




# Now you have cluster labels for each cell, which you can use to process loss for each cluster
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        positions = model(X)  # Shape: (num_nodes, 2)
        positions_numpy = positions.detach().cpu().numpy()
        x_pred, y_pred = positions[:, 0], positions[:, 1]

        # Compute total loss with all components
        loss = total_loss(x_pred, y_pred, hyperedges, area_info, cell_area,
                          overlap_threshold, bin_size, density_threshold, alpha = 10.0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 1 == 0:
            hpwl = smooth_hpwl(positions, hyperedges).item()
            density = soft_density_loss(x_pred, y_pred, area_info, cell_area, bin_size, density_threshold).item()
            validity = validity_penalty(x_pred, y_pred, area_info).item()


            print(f"Epoch {epoch}: HPWL={hpwl:.2f}, Density={density:.4f}, Validity={validity:.4f}")

In [16]:
in_dim = X.shape[1]
hidden_dim = 128
model = HyperGNN(in_dim, hidden_dim)

# Extract area info from net features or directly provide
area_info = {
    'lx': 2280,
    'ux': 70680,
    'ly': 2800,
    'uy': 70000
}

# Cell area (if uniform for all cells)
cell_area = torch.tensor(100000.0)  # Example value, replace with actual average area if known

# Grid size isn't needed directly now since we're using bin size
# But if needed elsewhere: total_area = (area_info['ux'] - area_info['lx']) * (area_info['uy'] - area_info['ly'])

# Train
train(
    model=model,
    X=X,
    hyperedges=hyperedges,
    area_info=area_info,
    cell_area=cell_area,
    grid_size=None,  # No longer used but retained for compatibility
    lambda_validity=1.0,
    lambda_overlap=0.1,
    lambda_hpwl=1.0,
    lambda_density=0.1,
    overlap_threshold=0.5,
    bin_size=100,
    density_threshold=0.8,
    num_epochs=100,
    lr=0.005

)

x range: -802382.5 -394.8228759765625
y range: -2552.83984375 914033.1875
Epoch 0: HPWL=194265088.00, Density=872764.3750, Validity=198629120.0000
x range: -193436.421875 -169.24659729003906
y range: -2564.568359375 301740.0
Epoch 1: HPWL=56221360.00, Density=1412667.6250, Validity=38848112.0000
x range: 30.6027889251709 365497.25
y range: -195099.21875 -267.41107177734375
Epoch 2: HPWL=63096508.00, Density=1241102.2500, Validity=46858776.0000
x range: 162.7744903564453 545350.0
y range: -94851.5625 -141.69630432128906
Epoch 3: HPWL=72097280.00, Density=974609.7500, Validity=54983024.0000
x range: 253.4740753173828 507794.21875
y range: 18.211463928222656 168891.09375
Epoch 4: HPWL=76308296.00, Density=1774651.0000, Validity=39201804.0000
x range: 308.48992919921875 265874.875
y range: 156.7173614501953 267408.90625
Epoch 5: HPWL=60030380.00, Density=2500647.2500, Validity=18679466.0000
x range: -88108.0703125 3173.781494140625
y range: 252.9799041748047 132697.0
Epoch 6: HPWL=25172878

In [17]:
X[0]

tensor([6.6270e+04, 1.4000e+02, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00])